In [1]:
!pip install datapackage-pipelines

     |████████████████████████████████| 102 kB 9.7 MB/s 
     |████████████████████████████████| 427 kB 36.6 MB/s 
     |████████████████████████████████| 72 kB 513 kB/s 
     |████████████████████████████████| 214 kB 51.0 MB/s 
     |████████████████████████████████| 327 kB 39.0 MB/s 
     |████████████████████████████████| 85 kB 4.9 MB/s 
     |████████████████████████████████| 68 kB 7.0 MB/s 
     |████████████████████████████████| 226 kB 57.8 MB/s 
     |████████████████████████████████| 58 kB 5.8 MB/s 
     |████████████████████████████████| 72 kB 546 kB/s 
     |████████████████████████████████| 184 kB 36.0 MB/s 
     |████████████████████████████████| 89 kB 8.7 MB/s 
     |████████████████████████████████| 48 kB 5.4 MB/s 
     |████████████████████████████████| 45 kB 3.4 MB/s 
     |████████████████████████████████| 88 kB 7.2 MB/s 
     |████████████████████████████████| 242 kB 52.7 MB/s 
     |████████████████████████████████| 131 kB 50.5 MB/s 
     |███████████████████████████

In [1]:
!pip install datapackage-pipelines-fiscal

     |████████████████████████████████| 119 kB 62.4 MB/s 
  Created wheel for datapackage-pipelines-aws: filename=datapackage_pipelines_aws-0.0.23-py2.py3-none-any.whl size=6805 sha256=48d08ae89a2e22bae6e799e7957116e433b3b30360bb745f76d2f9e2ac539773
  Stored in directory: /root/.cache/pip/wheels/dd/41/30/86d4c53dd8c9b66a31758054bc979d9a72fd853ce4db87ed5a
Successfully built datapackage-pipelines-aws


In [2]:
!echo $DPP_DB_ENGINE='22222'

=22222


In [7]:
#encoding: utf8
import os
import csv
import gzip
import sys
from datapackage_pipelines.wrapper import ingest, spew
import json
import copy
from datapackage_pipelines.generators import slugify

In [4]:
## Load datapackage
datapackage = json.load(open('mexico.json'))

In [5]:
resources = datapackage.get('resources', [])
original_resources = copy.deepcopy(resources)

In [10]:
for data_source in resources[0]['sources']:
    if data_source['url'].endswith('.csv'):
      print(data_source['mediatype'])
        # data_source['mediatype'] = 'text/csv'
    # if 'name' not in data_source:
    #     data_source['name'] = slugify(
    #         os.path.basename(data_source['url']),
    #         separator='_'
    #     ).lower()

TypeError: ignored

In [11]:
resources[0]["sources"]

KeyError: ignored

In [13]:
from hashlib import md5

from slugify import Slugify


slugger = Slugify(separator="_", safe_chars="-.")

In [18]:
title1 = "mexico"
owner_id1 = "R_kgDOGLhFbw"

In [19]:
def extract_names(source):
    title = source['title']
    dataset_name = source.get('name', title)
    dataset_name = slugger(dataset_name).lower()
    resource_name = source.get('resource-name', dataset_name)

    return title, dataset_name, resource_name


def extract_storage_ids(source):
    owner_id = source['id']
    _, dataset_name, _ = extract_names(source)
    dataset_id = '{}:{}'.format(owner_id, dataset_name)
    dataset_path = '{}/{}'.format(owner_id, dataset_name)
    dataset_name_hash = md5(dataset_name.encode('ascii')).hexdigest()[:16]
    db_table = '{}{}'.format(owner_id, dataset_name_hash)
    return dataset_id, db_table, dataset_path

In [22]:
for source in resources[:1]:
 print(extract_storage_ids(source))

('R_kgDOGLhFbw:presupuesto_mexico_2008.csv', 'R_kgDOGLhFbwf315a4b82f491b09', 'R_kgDOGLhFbw/presupuesto_mexico_2008.csv')


In [30]:
for source in resources[:1]:
  title, dataset_name, resource_name = extract_names(source)
  dataset_id, db_table, _ = extract_storage_ids(source)

  print(dataset_id,db_table)


R_kgDOGLhFbw:presupuesto_mexico_2008.csv R_kgDOGLhFbwf315a4b82f491b09


In [35]:
extract_storage_ids(datapackage)

('R_kgDOGLhFbw:mexico', 'R_kgDOGLhFbw4edfc924721abb77', 'R_kgDOGLhFbw/mexico')

In [37]:
field_types = dict((x['slug'], x['type'])
                       for x in datapackage['resources'][-1]['schema']['fields'])

In [38]:
field_types

{'CICLO': 'integer',
 'DESC_AI': 'string',
 'DESC_CAPITULO': 'string',
 'DESC_CONCEPTO': 'string',
 'DESC_FF': 'string',
 'DESC_FUNCION': 'string',
 'DESC_GPO_FUNCIONAL': 'string',
 'DESC_MODALIDAD': 'string',
 'DESC_PARTIDA_ESPECIFICA': 'string',
 'DESC_PARTIDA_GENERICA': 'string',
 'DESC_PP': 'string',
 'DESC_RAMO': 'string',
 'DESC_SUBFUNCION': 'string',
 'DESC_TIPOGASTO': 'string',
 'DESC_UR': 'string',
 'ENTIDAD_FEDERATIVA': 'string',
 'GPO_FUNCIONAL': 'string',
 'ID_AI': 'string',
 'ID_CAPITULO': 'string',
 'ID_CLAVE_CARTERA': 'string',
 'ID_CONCEPTO': 'string',
 'ID_ENTIDAD_FEDERATIVA': 'string',
 'ID_FF': 'string',
 'ID_FUNCION': 'string',
 'ID_MODALIDAD': 'string',
 'ID_PARTIDA_ESPECIFICA': 'string',
 'ID_PARTIDA_GENERICA': 'string',
 'ID_PP': 'string',
 'ID_RAMO': 'string',
 'ID_SUBFUNCION': 'string',
 'ID_TIPOGASTO': 'string',
 'ID_UR': 'string',
 'MONTO_ADEFAS': 'number',
 'MONTO_APROBADO': 'number',
 'MONTO_DEVENGADO': 'number',
 'MONTO_EJERCICIO': 'number',
 'MONTO_EJERCI

In [49]:
bbg_hierarchies = {}
bbg_dimensions = {}
bbg_measures = {}

In [52]:
from datapackage_pipelines.generators import slugify

In [43]:
model = datapackage["resources"][0]["model"]

In [46]:
ID_COLUMN_NAME = '_fdp__id_'

In [54]:
dataset_id, db_table, _ = extract_storage_ids(datapackage)

In [55]:
dataset_id, db_table

('R_kgDOGLhFbw:mexico', 'R_kgDOGLhFbw4edfc924721abb77')

In [56]:
## db_tables
source = datapackage["resources"][0]["schema"]
kinds = sorted(set(
      f['columnType'].split(':')[0]
      for f in source['fields']
  ) - {'value'})

resources = [
    slugify(kind, separator='_')
    for kind in kinds
]

db_tables = dict(
    (res, '{}_{}'.format(db_table, i))
    for i, res in enumerate(resources)
)
db_tables[''] = db_table

In [57]:
db_tables

{'': 'R_kgDOGLhFbw4edfc924721abb77',
 'activity': 'R_kgDOGLhFbw4edfc924721abb77_0',
 'administrative_classification': 'R_kgDOGLhFbw4edfc924721abb77_1',
 'budget_line_id': 'R_kgDOGLhFbw4edfc924721abb77_2',
 'date': 'R_kgDOGLhFbw4edfc924721abb77_3',
 'economic_classification': 'R_kgDOGLhFbw4edfc924721abb77_4',
 'expenditure_type': 'R_kgDOGLhFbw4edfc924721abb77_5',
 'fin_source': 'R_kgDOGLhFbw4edfc924721abb77_6',
 'functional_classification': 'R_kgDOGLhFbw4edfc924721abb77_7',
 'geo_source': 'R_kgDOGLhFbw4edfc924721abb77_8'}

In [60]:
for hierarchy_name, h_props in model['dimensions'].items():
  hierarchy_name = slugify(hierarchy_name, separator='_')
  hierarchy = dict(
      label=hierarchy_name,
      levels=h_props['primaryKey']
  )
  bbg_hierarchies[hierarchy_name] = hierarchy

  # Get all hierarchy columns
  attributes = h_props['attributes']
  attributes = list(attributes.items())

  # Separate to codes and labels
  codes = dict(filter(lambda x: 'labelfor' not in x[1], attributes))
  labels = dict(map(lambda y: (y[1]['labelfor'], y[1]),
                    filter(lambda x: 'labelfor' in x[1], attributes)))
  
  # For each code, create a babbage dimension
  for fieldname, attribute in codes.items():
      dimension_name = fieldname

      bbg_attributes = {
          fieldname: dict(
              column='.'.join([db_tables[hierarchy_name], fieldname]),
              label=attribute.get('title', attribute['source']),
              type=field_types[fieldname]
          )
      }
      bbg_dimension = dict(
          attributes=bbg_attributes,
          key_attribute=fieldname,
          label=attribute.get('title'),
          join_column=[hierarchy_name+'_id', ID_COLUMN_NAME]
      )

      label = labels.get(fieldname)
      if label is not None:
          fieldname = label['source']
          attribute = label
          bbg_attributes.update({
              fieldname: dict(
                  column='.'.join([db_tables[hierarchy_name],
                                  fieldname]),
                  label=attribute.get('title', attribute['source']),
                  type=field_types[fieldname]
              )
          })
          bbg_dimension.update(dict(
              label_attribute=fieldname
          ))
      bbg_dimensions[dimension_name] = bbg_dimension

  for measurename, measure in model['measures'].items():
        bbg_measures[measurename] = dict(
            column=measurename,
            label=measure.get('title', attribute['source']),
            type=field_types[measurename]
        )

  datapackage['babbageModel'] = dict(
      fact_table=db_tables[''],
      dimensions=bbg_dimensions,
      hierarchies=bbg_hierarchies,
      measures=bbg_measures
  )

In [62]:
import json

In [64]:
json.dumps(datapackage['babbageModel'])

'{"fact_table": "R_kgDOGLhFbw4edfc924721abb77", "dimensions": {"CICLO": {"attributes": {"CICLO": {"column": "R_kgDOGLhFbw4edfc924721abb77_3.CICLO", "label": "CICLO", "type": "integer"}}, "key_attribute": "CICLO", "label": "CICLO", "join_column": ["date_id", "_fdp__id_"]}, "GPO_FUNCIONAL": {"attributes": {"GPO_FUNCIONAL": {"column": "R_kgDOGLhFbw4edfc924721abb77_7.GPO_FUNCIONAL", "label": "GPO_FUNCIONAL", "type": "string"}, "DESC_GPO_FUNCIONAL": {"column": "R_kgDOGLhFbw4edfc924721abb77_7.DESC_GPO_FUNCIONAL", "label": "DESC_GPO_FUNCIONAL", "type": "string"}}, "key_attribute": "GPO_FUNCIONAL", "label": "GPO_FUNCIONAL", "join_column": ["functional_classification_id", "_fdp__id_"], "label_attribute": "DESC_GPO_FUNCIONAL"}, "ID_AI": {"attributes": {"ID_AI": {"column": "R_kgDOGLhFbw4edfc924721abb77_7.ID_AI", "label": "ID_AI", "type": "string"}, "DESC_AI": {"column": "R_kgDOGLhFbw4edfc924721abb77_7.DESC_AI", "label": "DESC_AI", "type": "string"}}, "key_attribute": "ID_AI", "label": "ID_AI", "j